In [2]:
!pip install spectral
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from google.colab import drive, files
import spectral.io.envi as envi
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
pip install --upgrade tensorflow


In [4]:
drive.mount('/content/drive')
#change this to your DS location
PATH_DATA = '/content/drive/MyDrive/ComputerVision/HyperBlood/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
IMAGES = ['B(1)','C(1)','D(1)','E(1)','E(7)','E(21)','F(1)','F(1a)','F(1s)','F(2)','F(2k)','F(7)','F(21)']


In [6]:
def get_data(name,remove_bands=True,clean=True):
    """
    Returns HSI data from a datacube

    Parameters:
    ---------------------
    name: name
    remove_bands: if True, noisy bands are removed (leaving 113 bands)
    clean: if True, remove damaged line

    Returns:
    -----------------------
    data, wavelenghts as numpy arrays (float32)
    """
    name = convert_name(name)
    filename = "{}data/{}".format(PATH_DATA,name)
    hsimage = envi.open('{}.hdr'.format(filename),'{}.float'.format(filename))
    wavs = np.asarray(hsimage.bands.centers)
    data = np.asarray(hsimage[:,:,:],dtype=np.float32)

    #removal of damaged sensor line
    if clean and name!='F_2k':
        data = np.delete(data,445,0)


    if not remove_bands:
        return data,wavs
    return data[:,:,get_good_indices(name)],wavs[get_good_indices(name)]

In [7]:
def get_anno(name,remove_uncertain_blood=True,clean=True, path=PATH_DATA):
    """
    Returns annotation (GT) for data files as 2D int numpy array
    Classes:
    0 - background
    1 - blood
    2 - ketchup
    3 - artificial blood
    4 - beetroot juice
    5 - poster paint
    6 - tomato concentrate
    7 - acrtylic paint
    8 - uncertain blood

    Parameters:
    ---------------------
    name: name
    clean: if True, remove damaged line
    remove_uncertain_blood: if True, removes class 8

    Returns:
    -----------------------
    annotation as numpy 2D array
    """
    name = convert_name(name)
    filename = "{}anno/{}".format(path,name)
    anno = np.load(filename+'.npz')['gt']
    #removal of damaged sensor line
    if clean and name!='F_2k':
        anno = np.delete(anno,445,0)
    #remove uncertain blood + technical classes
    if remove_uncertain_blood:
        anno[anno>7]=0
    else:
        anno[anno>8]=0

    return anno

In [8]:

def get_good_indices(name=None):
    """
    Returns indices of bands which are not noisy

    Parameters:
    ---------------------
    name: name
    Returns:
    -----------------------
    numpy array of good indices
    """
    name = convert_name(name)
    if name!='F_2k':
        indices = np.arange(128)
        indices = indices[5:-7]
    else:
        indices = np.arange(116)
    indices=np.delete(indices,[43,44,45])
    return indices

In [9]:
def convert_name(name):
    """
    Ensures that the name is in the filename format
    Parameters:
    ---------------------
    name: name

    Returns:
    -----------------------
    cleaned name
    """
    name = name.replace('(','_')
    name = name.replace(')','')
    return name

In [10]:
def preprocess_image(image_path, desired_width=224, desired_height=224):
    # Load the image
    image = Image.open(image_path)

    # Resize the image to match the input size expected by your models
    image = image.resize((desired_width, desired_height))

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Normalize pixel values to be in the range [0, 1]
    normalized_image = image_array / 255.0

    # Add any additional preprocessing steps based on your model requirements

    return normalized_image

In [11]:
def preprocess_data(data):
    pca = PCA(n_components=10)  # Reduce to 10 principal components
    flattened_data = data.reshape((data.shape[0] * data.shape[1], data.shape[2]))
    reduced_data = pca.fit_transform(flattened_data)
    preprocessed_data = reduced_data.reshape((data.shape[0], data.shape[1], pca.n_components))
    return preprocessed_data

In [12]:
# Load HSI data and annotations
HSI_data = []
HSI_anno = []

# Determine the maximum size along axis 1 for both data and annotations
max_size_axis_1_data = 0
max_size_axis_1_anno = 0

for img_name in IMAGES:
    data, _ = get_data(img_name)
    HSI_data.append(data)
    HSI_anno.append(np.ones(data.shape[:2]))  # Placeholder, replace with actual annotations

    # Update max size along axis 1 for data if needed
    max_size_axis_1_data = max(max_size_axis_1_data, data.shape[1])

    # Update max size along axis 1 for annotations if needed
    max_size_axis_1_anno = max(max_size_axis_1_anno, HSI_anno[-1].shape[1])

# Pad the arrays along axis 1 to have the same size for both data and annotations
HSI_data_padded = [np.pad(data, ((0, 0), (0, max_size_axis_1_data - data.shape[1]), (0, 0)), mode='constant') for data in HSI_data]
HSI_anno_padded = [np.pad(anno, ((0, 0), (0, max_size_axis_1_anno - anno.shape[1])), mode='constant') for anno in HSI_anno]

# Concatenate the padded arrays
HSI_data = np.concatenate(HSI_data_padded, axis=0)
HSI_anno = np.concatenate(HSI_anno_padded, axis=0)


In [13]:
HSI_data_preprocessed = preprocess_data(HSI_data)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(HSI_data_preprocessed, HSI_anno, test_size=0.3, random_state=42)


In [15]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (4889, 696, 10)
y_train shape: (4889, 696)
X_test shape: (2096, 696, 10)
y_test shape: (2096, 696)


In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming X_train and X_test are the preprocessed data
# and y_train, y_test are binary labels

# Reshape input data to (height, width, channels)
input_shape = (696, 10, 1)

# CNN Model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

# Assuming y_train and y_test are originally (batch_size, 696)
# Reshape them to (batch_size, 1)
y_train_binary = y_train[:, :1]
y_test_binary = y_test[:, :1]

# Compile and train the CNN model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train_binary, epochs=5, batch_size=128, validation_data=(X_test, y_test_binary))


Epoch 1/5
39/39 [==============================] - 3s 23ms/step - loss: 0.0141 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
39/39 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
39/39 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
39/39 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
39/39 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [26]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Assuming X_train and X_test are your preprocessed data
# and y_train_binary, y_test_binary are your binary labels

# LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(units=128, input_shape=(696, 10)))  # Adjust units and input shape as needed
lstm_model.add(Dense(units=1, activation='sigmoid'))  # Output layer for binary classification

# Compile and train the LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train_binary, epochs=5, batch_size=128, validation_data=(X_test, y_test_binary))


Epoch 1/5
39/39 [==============================] - 6s 62ms/step - loss: 0.1037 - accuracy: 0.9771 - val_loss: 9.9345e-04 - val_accuracy: 1.0000
Epoch 2/5
39/39 [==============================] - 2s 46ms/step - loss: 5.4647e-04 - accuracy: 1.0000 - val_loss: 3.0050e-04 - val_accuracy: 1.0000
Epoch 3/5
39/39 [==============================] - 2s 46ms/step - loss: 2.1837e-04 - accuracy: 1.0000 - val_loss: 1.5460e-04 - val_accuracy: 1.0000
Epoch 4/5
39/39 [==============================] - 2s 46ms/step - loss: 1.2401e-04 - accuracy: 1.0000 - val_loss: 9.7779e-05 - val_accuracy: 1.0000
Epoch 5/5
39/39 [==============================] - 2s 48ms/step - loss: 8.3150e-05 - accuracy: 1.0000 - val_loss: 6.9637e-05 - val_accuracy: 1.0000


In [25]:
# Evaluate the models
cnn_results = cnn_model.evaluate(X_test, y_test)
lstm_results = lstm_model.evaluate(X_test, y_test)

# Display evaluation metrics
print("\nCNN Evaluation:")
print(f"Loss: {cnn_results[0]}")
print(f"Accuracy: {cnn_results[1]}")

print("\nLSTM Evaluation:")
print(f"Loss: {lstm_results[0]}")
print(f"Accuracy: {lstm_results[1]}")

66/66 [==============================] - 1s 12ms/step - loss: 0.0824 - accuracy: 0.9937

CNN Evaluation:
Loss: 0.9816523194313049
Accuracy: 0.993693470954895

LSTM Evaluation:
Loss: 0.08241873979568481
Accuracy: 0.993693470954895
